<a href="https://www.kaggle.com/code/inhngcn/cinnamon-week5?scriptVersionId=138386152" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [24]:
!git clone https://github.com/andythetechnerd03/cinnamon-week5.git

Cloning into 'cinnamon-week5'...
remote: Enumerating objects: 1212, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1212 (delta 116), reused 166 (delta 109), pack-reused 1035
Receiving objects: 100% (1212/1212), 143.61 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (765/765), done.


In [25]:
%cd cinnamon-week5/

/kaggle/working/cinnamon-week5/cinnamon-week5/cinnamon-week5


In [26]:
!nvidia-smi

Sun Jul 30 11:05:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    29W /  70W |   1490MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

## Information

### Hardware

- **Platform:** Kaggle Notebook
- **Framework**: PyTorch 2.0.0
- **GPU**: 2x Nvidia Tesla T4 (16GB Memory)
- **CUDA**: 11.4

### Model

- I chose the GoogLemodel model with 6.2M parameters.
- Reason: I like the name because it has "Google" in it. And it is rather lightweight compared to other models like Resmodel while still maintaining the performace in inference.

## Structured Pruning

In [27]:
!pip install --upgrade torch_pruning

In [28]:
import torch
import torchvision
import torch_pruning as tp

In [ ]:
from models.googlenet import googlenet

model = googlenet()
model.load_state_dict(torch.load('/kaggle/working/cinnamon-week5/googlenet-196-best.pth'))
model

In [30]:
imp = tp.importance.MagnitudeImportance(p=2)

ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 100: # ignore the classifier
        ignored_layers.append(m)

round_to = None
if isinstance(model, torchvision.models.vision_transformer.VisionTransformer):
    round_to = model.encoder.layers[0].num_heads # model-specific restriction

pruner = tp.pruner.MagnitudePruner(
    model = model,
    example_inputs = torch.randn(1,3,224,224),
    importance = imp,     # Importance Estimator
    global_pruning=False, # Please refer to Page 9 of https://www.cs.princeton.edu/courses/archive/spring21/cos598D/lectures/pruning.pdf
    ch_sparsity = 0.2,    # global sparsity for all layers
    #ch_sparsity_dict = {model.conv1: 0.2}, # manually set the sparsity of model.conv1
    iterative_steps = 1,  # number of steps to achieve the target ch_sparsity.
    ignored_layers = ignored_layers,        # ignore some layers such as the finall linear classifier
    round_to = round_to,  # round channels
    #unwrapped_parameters=[ (model.features[1][1].layer_scale, 0), (model.features[5][4].layer_scale, 0) ],
)

In [31]:
# Model size before pruning
base_macs, base_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,32,32))
pruner.step()

# modify some inferece-related attributes if necessary
if isinstance(
    model, torchvision.models.vision_transformer.VisionTransformer
):  # ViT relies on the hidden_dim attribute for forwarding, so we have to modify this variable after pruning
    model.hidden_dim = model.conv_proj.out_channels

# Parameter & MACs Counter
pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,32,32))
print("The pruned model:")
print(model)
print("Summary:")
print("Params: {:.2f} M => {:.2f} M".format(base_nparams/1e6, pruned_nparams/1e6))
print("MACs: {:.2f} G => {:.2f} G".format(base_macs/1e9, pruned_macs/1e9))

# Test Forward
output = model(torch.randn(1,3,32,32))
print("Output.shape: ", output.shape)

# Test Backward
loss = torch.nn.functional.cross_entropy(output, torch.randint(1, 100, (1,)))
loss.backward()

model = model.cuda()

The pruned model:
GoogleNet(
  (prelayer): Sequential(
    (0): Conv2d(3, 51, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(51, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(51, 51, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(51, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(51, 153, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(153, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (a3): Inception(
    (b1): Sequential(
      (0): Conv2d(153, 51, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(51, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (b2): Sequential(
      (0): Conv2d(153, 76, kernel_size=(1, 1), stride=(1, 1))
      (1): Bat

## Training the model

In [49]:
# train.py
# !/usr/bin/env	python3

""" train modelwork using pytorch

author baiyu
"""

import os
import sys
import argparse
import time
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from conf import settings
from utils import get_training_dataloader, get_test_dataloader, WarmUpLR, \
    most_recent_folder, most_recent_weights, last_epoch, best_acc_weights

model = model.cuda()
lr = 0.001
warm = 1
b = 128
gpu = True
resume = False

def train(epoch):
    start = time.time()
    model.train()
    for batch_index, (images, labels) in enumerate(cifar100_training_loader):

        if gpu:
            labels = labels.cuda()
            images = images.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        n_iter = (epoch - 1) * len(cifar100_training_loader) + batch_index + 1

        last_layer = list(model.children())[-1]
        for name, para in last_layer.named_parameters():
            if 'weight' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_weights', para.grad.norm(), n_iter)
            if 'bias' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_bias', para.grad.norm(), n_iter)

        '''print('Training Epoch: {epoch} [{trained_samples}/{total_samples}]\tLoss: {:0.4f}\tLR: {:0.6f}'.format(
            loss.item(),
            optimizer.param_groups[0]['lr'],
            epoch=epoch,
            trained_samples=batch_index * b + len(images),
            total_samples=len(cifar100_training_loader.dataset)
        ))'''

        # update training loss for each iteration
        writer.add_scalar('Train/loss', loss.item(), n_iter)

        if epoch <= warm:
            warmup_scheduler.step()

    for name, param in model.named_parameters():
        layer, attr = os.path.splitext(name)
        attr = attr[1:]
        writer.add_histogram("{}/{}".format(layer, attr), param, epoch)

    finish = time.time()

    print('epoch {} training time consumed: {:.2f}s'.format(epoch, finish - start))


@torch.no_grad()
def eval_training(epoch=0, tb=True):
    start = time.time()
    model.eval()

    test_loss = 0.0  # cost function error
    correct = 0.0

    for (images, labels) in cifar100_test_loader:

        if gpu:
            images = images.cuda()
            labels = labels.cuda()

        outputs = model(images)
        loss = loss_function(outputs, labels)

        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

    finish = time.time()
    if gpu:
        print('GPU INFO.....')
        print(torch.cuda.memory_summary(), end='')
    print('Evaluating modelwork.....')
    print('Test set: Epoch: {}, Average loss: {:.4f}, Accuracy: {:.4f}, Time consumed:{:.2f}s'.format(
        epoch,
        test_loss / len(cifar100_test_loader.dataset),
        correct.float() / len(cifar100_test_loader.dataset),
        finish - start
    ))
    print()

    # add informations to tensorboard
    if tb:
        writer.add_scalar('Test/Average loss', test_loss / len(cifar100_test_loader.dataset), epoch)
        writer.add_scalar('Test/Accuracy', correct.float() / len(cifar100_test_loader.dataset), epoch)

    return correct.float() / len(cifar100_test_loader.dataset)




# data preprocessing:
cifar100_training_loader = get_training_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=4,
    batch_size=128,
    shuffle=True
)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=settings.MILESTONES,
                                                 gamma=0.2)  # learning rate decay
iter_per_epoch = len(cifar100_training_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * warm)

if resume:
    recent_folder = most_recent_folder(os.path.join(settings.CHECKPOINT_PATH, model), fmt=settings.DATE_FORMAT)
    if not recent_folder:
        raise Exception('no recent folder were found')

    checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, model, recent_folder)

else:
    checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, 'model', settings.TIME_NOW)

# use tensorboard
if not os.path.exists(settings.LOG_DIR):
    os.mkdir(settings.LOG_DIR)

# since tensorboard can't overwrite old values
# so the only way is to create a new tensorboard log
writer = SummaryWriter(log_dir=os.path.join(
    settings.LOG_DIR, 'model', settings.TIME_NOW))
input_tensor = torch.Tensor(1, 3, 32, 32)
if gpu:
    input_tensor = input_tensor.cuda()
writer.add_graph(model, input_tensor)

# create checkpoint folder to save model
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{model}-{epoch}-{type}.pth')

best_acc = 0.0
if resume:
    best_weights = best_acc_weights(os.path.join(settings.CHECKPOINT_PATH, model, recent_folder))
    if best_weights:
        weights_path = os.path.join(settings.CHECKPOINT_PATH, model, recent_folder, best_weights)
        print('found best acc weights file:{}'.format(weights_path))
        print('load best training file to test acc...')
        model.load_state_dict(torch.load(weights_path))
        best_acc = eval_training(tb=False)
        print('best acc is {:0.2f}'.format(best_acc))

    recent_weights_file = most_recent_weights(os.path.join(settings.CHECKPOINT_PATH, model, recent_folder))
    if not recent_weights_file:
        raise Exception('no recent weights file were found')
    weights_path = os.path.join(settings.CHECKPOINT_PATH, model, recent_folder, recent_weights_file)
    print('loading weights file {} to resume training.....'.format(weights_path))
    model.load_state_dict(torch.load(weights_path))

    resume_epoch = last_epoch(os.path.join(settings.CHECKPOINT_PATH, model, recent_folder))

for epoch in range(1, 1 + 1):
    if epoch > warm:
        train_scheduler.step(epoch)

    if resume:
        if epoch <= resume_epoch:
            continue

    train(epoch)
    acc = eval_training(epoch)

    # start to save best performance model after learning rate decay to 0.01
    if epoch > settings.MILESTONES[1] and best_acc < acc:
        weights_path = checkpoint_path.format(model='model', epoch=epoch, type='best')
        print('saving weights file to {}'.format(weights_path))
        torch.save(model.state_dict(), weights_path)
        best_acc = acc
        continue

    if not epoch % settings.SAVE_EPOCH:
        weights_path = checkpoint_path.format(model='model', epoch=epoch, type='regular')
        print('saving weights file to {}'.format(weights_path))
        torch.save(model.state_dict(), weights_path)



Files already downloaded and verified
epoch 1 training time consumed: 57.92s
GPU INFO.....
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1892 MiB |   3317 MiB |    943 TiB |    943 TiB |
|       from large pool |   1853 MiB |   3276 MiB |    928 TiB |    928 TiB |
|       from small pool |     39 MiB |     80 MiB |     14 TiB |     14 TiB |
|---------------------------------------------------------------------------|
| Active memory         |   1892 MiB |   3317 MiB |

## Testing the model

In [50]:
torch.save(model.state_dict(), '/kaggle/working/cinnamon-week5/structured_pruned_model.pth')

In [57]:
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
import argparse

from matplotlib import pyplot as plt

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from conf import settings
from utils import get_test_dataloader
cifar100_test_loader = get_test_dataloader(
        settings.CIFAR100_TRAIN_MEAN,
        settings.CIFAR100_TRAIN_STD,
        #settings.CIFAR100_PATH,
        num_workers=4,
        batch_size=16,
    )
correct_1 = 0.0
correct_5 = 0.0
total = 0
total_time = 0

with torch.no_grad():
    for n_iter, (image, label) in enumerate(cifar100_test_loader):
        print("iteration: {}\ttotal {} iterations".format(n_iter + 1, len(cifar100_test_loader)))

        image = image.cuda()
        label = label.cuda()
            #print('GPU INFO.....')
            #print(torch.cuda.memory_summary(), end='')

        #if model == 'quantized_googlemodel':
        #    qm(image)
        # Measure time
        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        starter.record()
        with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
            with record_function("model_inference"):
                output = model(image)

        ender.record()
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)/10000
        total_time += curr_time

        _, pred = output.topk(5, 1, largest=True, sorted=True)

        label = label.view(label.size(0), -1).expand_as(pred)
        correct = pred.eq(label).float()

        #compute top 5
        correct_5 += correct[:, :5].sum()

        #compute top1
        correct_1 += correct[:, :1].sum()

print('GPU INFO.....')
print(torch.cuda.memory_summary(), end='')

print()
print("Top 1 err: ", 1 - correct_1 / len(cifar100_test_loader.dataset))
print("Top 5 err: ", 1 - correct_5 / len(cifar100_test_loader.dataset))
print(f"Average inference time per image: {total_time} miliseconds")
print("Parameter numbers: {}".format(sum(p.numel() for p in model.parameters())))
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


Files already downloaded and verified
iteration: 1	total 625 iterations


STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


iteration: 2	total 625 iterations


STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-30 15:35:03 28:28 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


iteration: 3	total 625 iterations


...

iteration: 624	total 625 iterations


STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


iteration: 625	total 625 iterations


STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-30 15:37:56 28:28 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


GPU INFO.....
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1108 MiB |   3317 MiB |    944 TiB |    944 TiB |
|       from large pool |   1069 MiB |   3276 MiB |    929 TiB |    929 TiB |
|       from small pool |     38 MiB |     80 MiB |     14 TiB |     14 TiB |
|---------------------------------------------------------------------------|
| Active memory         |   1108 MiB |   3317 MiB |    944 TiB |    944 TiB |
|       from large pool |   1069 MiB |   3276 MiB 

In [59]:
!python test.py -net googlenet -weights /kaggle/working/cinnamon-week5/pruned_googlenet.pth  -gpu

Files already downloaded and verified
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
GoogleNet(
  (prelayer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 192, kernel_size=(3, 3)

In [58]:
!python test.py -net googlenet -weights /kaggle/working/cinnamon-week5/googlenet-196-best.pth -gpu

Files already downloaded and verified
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
GoogleNet(
  (prelayer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 192, kernel_size=(3, 3)

# Quantization FP16 (Post Training Static Quantization)